<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
from omegaconf import OmegaConf
import torch
from pathlib import Path
from datetime import datetime
import gc, json, os, sys

%load_ext autoreload
%autoreload 2

from stable_diffusion_utils import load_model_from_config, config, generate

In [ ]:
torch.cuda.is_available()

In [ ]:
opt.ckpt = 'D:/models/stable_diffusion/sd-v1-4.ckpt'
opt.config = str(projects_dir / 'stable-diffusion/configs/stable-diffusion/v1-inference.yaml')

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
opt = config()
model = load_model_from_config(OmegaConf.load(f"{opt.config}"), f"{opt.ckpt}").to(device)
batch_idx = 0
sample_idx = 0

In [ ]:
outputs_path = Path.home() / 'Documents/generated_data/stable_diffusion'
batch_name = f'test' # name for subfolder and filenames  {datetime.now().strftime("%Y%m%d_%H%M%S")}_
width_height = [512, 512] 
opt.W = width_height[0]
opt.H = width_height[1]
opt.scale = 20.5 # strength of text prompt
opt.steps = 100 # number of diffusion steps
opt.n_samples = 1 # not exposed, you can do 2 or more based on GPU ram, if get CUDA out of memory need to restart runtime
opt.sampler = 'klms' # ["klms","plms", "ddim"]
opt.ddim_eta = 0.1 # scale of variance from 0.0 to 1.0
opt.seed = -1 # use -1 for random seed or specify number manually
opt.n_iter = 1
num_batch_images = 5 # how many images you want to generate in this batch

# Init image
opt.init_img = str(Path.home() / '') # url or path to an image, or path to a folder to pick random images from
opt.strength = 0.5 # from 0.0 to 1.0 how much the init image is used

# Mask
opt.mask_path = str(Path.home() / '')

# Prompt
opt.prompt = "" 

opt.outdir = str(outputs_path / batch_name)

opt.process_config()

# save settings
settings = vars(opt)
os.makedirs(opt.outdir, exist_ok=True)
while os.path.isfile(f"{opt.outdir}/{batch_name}_{batch_idx}_settings.txt"):
    batch_idx += 1
with open(f"{opt.outdir}/{batch_name}_{batch_idx}_settings.txt", "w+", encoding="utf-8") as f:
    json.dump(settings, f, ensure_ascii=False, indent=4)

sample_idx = 0
for i in range(num_batch_images):
    gc.collect()
    torch.cuda.empty_cache()

    images = generate(opt, model, batch_name, batch_idx, sample_idx)

    print(f"Saved to: {opt.outdir}")
    opt.seed += 1
    sample_idx += 1
